# feature analysis result - already exist in the previous result file

We use WL or CCA to do the
1. inter-subject gesture classification
2. Few-shot adaptation

WL로 가자

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, random
sys.path.append('../../Shared')
import processing, Model, Visualization, modules, Adaptation
from sklearn.model_selection import train_test_split



def import_data(subject, current_rep, db_name, feat_idx):
    path = f'C:/Users/hml76/Desktop/UMN/Research/5th Federated Prototype Learning/Dataset/{db_name}/'

    x_train = np.array(pd.read_csv(path + 'Sub{}/Sub{}_Rep{}_data.csv'.format(subject, subject, current_rep)))
    y_train = np.array(pd.read_csv(path + 'Sub{}/Sub{}_Rep{}_label.csv'.format(subject, subject, current_rep)))
    indices = np.random.permutation(len(x_train)) #shuffle
    x_train, y_train = x_train[indices], y_train[indices]

    x_train = x_train.reshape(len(x_train), 12, 11)
    x_train = x_train[:, :, feat_idx]
    x_train = x_train.reshape(len(x_train), 12, 1)

    return x_train, y_train

def ready_data_per_subject_all_reps(subject, Data, downsample, downsample_size, feat_idx):
    X, y = [], []
    for rep in range(1,7):
        X_tmp, y_tmp = import_data(subject, rep, Data, feat_idx)

        X.append(X_tmp)
        y.append(y_tmp)
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    y = np.array([np.argmax(y[i]) for i in range(len(y))])

    if downsample:
        idx = np.random.choice(X.shape[0], size=downsample_size, replace=False)
        X, y = X[idx], y[idx]

    return X, y

In [2]:
sub_lst = random.sample(range(1, 41), 20)
#sub_lst
sub_lst = [35, 28, 26, 23, 6, 15, 33, 39, 4, 14, 3, 16, 29, 17, 13, 9, 20, 38, 11, 25]

# Baseline - LOSO

Random 10 subjects

In [ ]:
Data_name = "DB2"
F = 0

Sub_all = []

for train_subject in sub_lst:
    print(f"Testing subject {train_subject}")
    X_train, y_train = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=True, downsample_size=25000, feat_idx=F)

    #model = Model.build_model_deep_1D(num_classes=len(np.unique(y_train)))
    model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
    history = model.fit(X_train, y_train, epochs=100, batch_size=256, verbose=0)
    plt.plot(history.history['accuracy'])
    plt.show()

    Sub_acc = []
    for test_subject in sub_lst:
        if test_subject != train_subject:
            X_test, y_test = ready_data_per_subject_all_reps(subject=test_subject, Data=Data_name, downsample=True, downsample_size=10000, feat_idx=F)
            loss, acc = model.evaluate(X_test, y_test, verbose=0)
            print(f"\t Testing subject {test_subject} : acc {acc*100:.2f}%")
            Sub_acc.append(acc)
        else:
            continue
    Sub_all.append(Sub_acc)

Testing subject 35


In [4]:
Sub_all

[[0.02239999920129776,
  0.011599999852478504,
  0.01730000041425228,
  0.038100000470876694,
  0.04280000180006027,
  0.04749999940395355,
  0.0272000003606081,
  0.037300001829862595,
  0.018799999728798866,
  0.021700000390410423,
  0.011699999682605267,
  0.00860000029206276,
  0.023900000378489494,
  0.017400000244379044,
  0.03449999913573265,
  0.03420000150799751,
  0.010900000110268593,
  0.00430000014603138,
  0.022299999371170998],
 [0.04390000179409981,
  0.034299999475479126,
  0.023399999365210533,
  0.023900000378489494,
  0.0421999990940094,
  0.06710000336170197,
  0.024299999698996544,
  0.03819999843835831,
  0.030700000002980232,
  0.025499999523162842,
  0.019999999552965164,
  0.030300000682473183,
  0.022199999541044235,
  0.03460000082850456,
  0.023399999365210533,
  0.025499999523162842,
  0.010300000198185444,
  0.008999999612569809,
  0.03880000114440918],
 [0.03720000013709068,
  0.03009999915957451,
  0.021700000390410423,
  0.026599999517202377,
  0.04030

In [5]:
baseline = pd.DataFrame(Sub_all)
baseline.shape

(20, 19)

In [6]:
baseline.to_csv('./DB2_baseline.csv', index=False)

# Inter-subject incremental

In [8]:
Data_name = "DB2"
F = 0
Sub_all = []

for test_subject in sub_lst:
    print(f"Testing subject {test_subject}")
    X_test, y_test = ready_data_per_subject_all_reps(subject=test_subject, Data=Data_name, downsample=True, downsample_size=10000, feat_idx=F)

    Sub_acc, X_train, y_train = [], [], []
    for train_subject in sub_lst:
        if train_subject != test_subject:
            X_train_tmp, y_train_tmp = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=True, downsample_size=25000, feat_idx=F)
            X_train.append(X_train_tmp)
            y_train.append(y_train_tmp)

            X_train_arr, y_train_arr = np.concatenate(X_train, axis=0), np.concatenate(y_train, axis=0)
            print("\t", X_train_arr.shape, y_train_arr.shape)

            #model = Model.build_model_deep_1D(num_classes=len(np.unique(y_train_arr)))
            model = Model.build_model_1D(=, num_classes=len(np.unique(y_train_arr)))
            model.fit(X_train_arr, y_train_arr, epochs=100, batch_size=256, verbose=0)
            loss, acc = model.evaluate(X_test, y_test, verbose=0)
            print(f"\t Added subject {train_subject} : acc {acc*100:.2f}%")
            Sub_acc.append(acc)

    Sub_all.append(Sub_acc)

Testing subject 35
	 (25000, 12, 1) (25000,)
	 Added subject 28 : acc 2.65%
	 (50000, 12, 1) (50000,)
	 Added subject 26 : acc 2.22%
	 (75000, 12, 1) (75000,)
	 Added subject 23 : acc 2.54%
	 (100000, 12, 1) (100000,)
	 Added subject 6 : acc 2.04%
	 (125000, 12, 1) (125000,)
	 Added subject 15 : acc 1.47%
	 (150000, 12, 1) (150000,)
	 Added subject 33 : acc 4.24%
	 (175000, 12, 1) (175000,)
	 Added subject 39 : acc 3.16%
	 (200000, 12, 1) (200000,)
	 Added subject 4 : acc 2.48%
	 (225000, 12, 1) (225000,)
	 Added subject 14 : acc 1.43%
	 (250000, 12, 1) (250000,)


KeyboardInterrupt: 

In [ ]:
acccc = pd.DataFrame(Sub_all)
acccc.to_csv('./DB2_inter_sub_accumulate.csv', index=False)
acccc.shape

# Few-shot adaptation